In [ ]:
%%capture
import os, re
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    import torch; v = re.match(r"[0-9\.]{3,}", str(torch.__version__)).group(0)
    xformers = "xformers==" + ("0.0.32.post2" if v == "2.8.0" else "0.0.29.post3")
    !pip install --no-deps bitsandbytes accelerate {xformers} peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1,<4.0.0" "huggingface_hub>=0.34.0" hf_transfer
    !pip install --no-deps unsloth

!pip install transformers==4.55.4
!pip install --no-deps trl==0.22.2
!pip install gradio

In [ ]:
#Importe das bibliotecas

from unsloth import FastLanguageModel, is_bfloat16_supported
import torch
from trl import SFTTrainer
from transformers import TrainingArguments
from transformers import TextStreamer
from datasets import load_dataset
from google.colab import userdata

In [ ]:
# Informações da GPU de processamento

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

> O modelo selecionado para receber o fine-tuning fo o Olhama de 1B de parâmetros. Ele funcionou bem na execução local e tinha um conjunto de respostas interessante para o nosso dataset de treinamento.

In [ ]:
# Variáveis locais

HF_TOKEN = userdata.get('HF_TOKEN')                       #Token do Google Colab
FINETUNING_MODEL = 'unsloth/Llama-3.2-1B-Instruct'        #modelo
FINETUNING_DATASET = 'Araguacy/amazon'                    #caminho do Hugging Face para o dataset
FINETUNED_MODEL = 'Araguacy/Llama-3.2-1B-finetuning-fiap' #Nome do modelo após treinamento
MAX_SEQ_LENGTH = 2048
LOAD_IN_4BIT = True #Parâmetro para o Unsloth usar a biblioteca bitsandbytes para carregar o modelo quantizado, economizando VRAM.
DTYPE = None

In [ ]:

# Download do modelo, primeiro passo

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = FINETUNING_MODEL,
    max_seq_length = MAX_SEQ_LENGTH,
    dtype=DTYPE,
    load_in_4bit= LOAD_IN_4BIT,   # Quantização 4 bit (comprime)
    token=HF_TOKEN
  )

> Realizando uma inferência no modelo base para teste passando um prompt para falar sobre Harlequin.

In [ ]:
FastLanguageModel.for_inference(model)
streamer = TextStreamer(tokenizer)
prompt = 'What do you know about the Harlequin?'
prompt_tokenizer = tokenizer(prompt, return_tensors='pt').to('cuda')

_=model.generate(**prompt_tokenizer, streamer=streamer, max_new_tokens=120)

In [ ]:
#Configuração do PEFT (LoRA), segundo passo
# r = 16: Define o "ranking" (tamanho) dos adaptadores LoRA.
# target_modules: Lista quais camadas do Transformer receberão os adaptadores.

model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

In [ ]:
#Função para formatação do dataset para ser enviado ao treinamento.

def format_llama3_template(example):
    messages = [
        {
            "role": "system",
            "content": "You are a product identification model. Your job is to analyze product questions and generate product information as a response.",
        },
        {
            "role": "user",
            "content": f"Describe the product: {example['title']}\n\nWhat do you know about the {example['title']}?",
        },
        {
            "role": "assistant",
            "content": example['content'],
        }
    ]

    # 2. Aplica o template de chat do Llama 3, garantindo o token EOS e a máscara de loss.
    # O "tokenize=False" garante que recebemos apenas a string formatada.
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=False
    )

    # Adiciona o token EOS, que é CRUCIAL para o SFT em modelos de chat
    return { "text": text + tokenizer.eos_token }

O dataset nesta etapa já consta com todas as melhorias avaliadas em outros datasets, com isso temos a leitura dos dados mais rápidas via hugging face.

In [ ]:
# Load do dataset pelo Hugging face

dataset = load_dataset(FINETUNING_DATASET)
train_dataset = dataset['train']

In [ ]:
# Aplica a formatação a todos os 100.000 registros
formatted_train_dataset = train_dataset.map(format_llama3_template)

In [ ]:
# Print resultado da string formatada
print(train_dataset)

In [ ]:
formatted_train_dataset[0]

>> Os parâmetros de treinamento possui um conjunto de configuração que mescla valores default definidos na documentação do unsloth com observações de execuções anteriores, principalmente por usar um modelo instruído que facilita o fine-tuning.
Com isso, o foco dos parâmetros é obter o melhor resultado possível com o menor tempo de execução.

In [ ]:
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=formatted_train_dataset,
    dataset_text_field="text",
    max_seq_length=2048,
    dataset_num_proc=4,
    packing = True,
    args = TrainingArguments(
        per_device_train_batch_size=16,
        gradient_accumulation_steps=2,
        warmup_steps=10,
        num_train_epochs=3,               #O modelo verá o dataset inteiro 3 vezes
        learning_rate=3e-5,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps=1,
        output_dir="outputs",
        optim="paged_adamw_8bit",        #Mais uma otimização de memória
        lr_scheduler_type = 'cosine',
        seed=42,
        #max_steps = 1 #removido para conseguir avaliar o treinamento do modelo com todos os dados. Está usando uma versão com
    ),
)

In [ ]:
# Treinamento do modelo, será necessário passar a chave do wandb (Weights & Biases) para monitorar o treinamento
trainer.train()

## Resultado do treinamento :

> TrainOutput(global_step=9375, training_loss=1.8145994728342691, metrics={'train_runtime': 7248.7627, 'train_samples_per_second': 41.386, 'train_steps_per_second': 1.293, 'total_flos': 6.901003531211244e+17, 'train_loss': 1.8145994728342691, 'epoch': 3.0})

* global_step=9375: Este é o número total de "passos de otimização" que o modelo executou. Cálculo: (100.000 exemplos / 32) * 3 épocas = 3.125 * 3 = 9.375 passos
* training_loss=1.8145: Este é o número mais importante para a qualidade do modelo. "Loss" (perda) é a medida de "erro" do modelo
* train_runtime: 7248.7627: O tempo total que o fine-tuning levou para rodar, em segundos. Ou seja, o processo todo demorou pouco mais de 2 horas na GPU A100.
* train_samples_per_second: 41.386:A velocidade do seu treinamento. A GPU conseguiu processar, em média, 41.3 exemplos do seu dataset por segundo.
* train_steps_per_second: 1.293:A velocidade de atualização. O modelo estava realizando 1.29 atualizações de peso (passos) por segundo.
* total_flos: 6.90...e+17: FLOS (Floating Point Operations) é o número total de cálculos matemáticos que a GPU realizou. É um número astronômico que apenas quantifica o esforço computacional total.

In [ ]:
# salvar os arquivos localmente em uma pasta
model.save_pretrained(FINETUNED_MODEL)
tokenizer.save_pretrained(FINETUNED_MODEL)

# 1. Este comando envia os adaptadores LoRA (o "modelo") para o Hub
model.push_to_hub(FINETUNED_MODEL, token = HF_TOKEN)

# 2. Este comando envia os arquivos do tokenizer para o Hub
tokenizer.push_to_hub(FINETUNED_MODEL, token = HF_TOKEN)

In [ ]:
#Salvar o modelo quantizado no Hugging Face
model.push_to_hub_gguf(FINETUNED_MODEL, tokenizer, quantization_method = "q8_0", token = HF_TOKEN)

In [ ]:
import gradio as gr

In [ ]:
model_base, _ = FastLanguageModel.from_pretrained(
    model_name = FINETUNING_MODEL,
    max_seq_length = MAX_SEQ_LENGTH,
    dtype=DTYPE,
    load_in_4bit= LOAD_IN_4BIT,
    token=HF_TOKEN
  )

In [ ]:
# Para nova inferência do modelo base
FastLanguageModel.for_inference(model_base)

In [ ]:
# Para inferência do modelo treinado
FastLanguageModel.for_inference(model)

In [ ]:
# Função para comparação/teste do modelo base x modelo treinado

def models_comparison(input = ""):
    messages = [
        {
            "role": "system",
            "content": "You are a product identification model. Your job is to analyze product questions and generate product information as a response.",
        },
        {
            "role": "user",
            "content": f"Describe the product: {input}\n\nWhat do you know about the {input}?",
        },
    ]

    inference_prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
    )

    inputs = tokenizer([inference_prompt], return_tensors="pt").to("cuda")

    ## Configuração para inferência
    gen_kwargs = {
        "max_new_tokens": 128,
        "do_sample": True,
        "temperature": 1,
        "repetition_penalty": 0.9,
    }

    # Gerado a partir do modelo base
    base_output_tokens = model_base.generate(**inputs, **gen_kwargs)
    base_response = tokenizer.batch_decode(base_output_tokens[:, inputs.input_ids.shape[1]:], skip_special_tokens=True)[0]

    # Gerado a partir do modelo finetuned
    ft_output_tokens = model.generate(**inputs, **gen_kwargs)
    ft_response = tokenizer.batch_decode(ft_output_tokens[:, inputs.input_ids.shape[1]:], skip_special_tokens=True)[0]

    return base_response, ft_response

In [ ]:
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("# 🤖 Comparação de Modelos: Llama 3.2 1B vs. Modelo com Fine-tuning")
    gr.Markdown("Digite um título de produto abaixo para ver como o Modelo Base e o Modelo com Fine-tuning respondem.")

    inp = gr.Textbox(label="Digite um Título (Input)", placeholder="Exemplo: O que você sabe sobre o produto?")
    btn = gr.Button("Gerar Respostas")

    with gr.Row():
        out_base = gr.Textbox(label="Resposta do Modelo Base", lines=10)
        out_ft = gr.Textbox(label="Resposta do Modelo com Fine-tuning", lines=10)

    btn.click(fn=models_comparison, inputs=inp, outputs=[out_base, out_ft])

# Cria um link para uma aplicação temporária para teste
demo.launch(share=True)